잡플래닛에서 기업의 평점을 수집해 가공해보기<br>
평가 점수와 날짜만 수집.<br>
(기회가 된다면 XPATH, CSS_SELECTOR 둘 중 하나로 통일하기) 

In [1]:
import selenium
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "chromedriver.exe")
service = Service(file_path)
driver = webdriver.Chrome(service=service)

In [2]:
corp = "SK하이닉스"
url = "https://www.jobplanet.co.kr/companies/20561/reviews/%EC%97%90%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4"
driver.get(url)
driver.implicitly_wait(10) # 페이지가 전부 로딩되기까지 최대 10초 대기

try:
    driver.find_element(By.CSS_SELECTOR, "#i8zsl").click()  # ID로 찾기
except:
    pass

In [3]:
# **이건 하드코딩이어서 리뷰 바뀌면 못따라간다. 언제나 2025.03 값을 도출함(하이닉스 리뷰).
review_list = driver.find_elements(By.CSS_SELECTOR, "#viewReviewsList")
review = review_list[0]
score_list = review.find_element(By.CSS_SELECTOR, r"#ReviewCardSide > div.mb-\[32px\] > span")
date = review.find_elements(By.CSS_SELECTOR, "#review_4987244 > div:nth-child(1) > div.relative.flex.items-center.justify-between > div > span:nth-child(7)")[0].get_attribute("innerHTML")[:-3]
print(date)

2025. 03


In [4]:
score_list = review.find_element(By.CSS_SELECTOR, "#ReviewCardSide")
star_score = score_list.find_element(By.CSS_SELECTOR, r"#ReviewCardSide > div.mb-\[32px\] > span").get_attribute("innerHTML")
display(star_score)

'5.0'

In [5]:
# 항목별 평점
bl_score_list = score_list.find_elements(By.CSS_SELECTOR, "#ReviewCardSide > div:nth-child(2) > ul")
# promotion_opp = len(score_list.find_element(By.XPATH, ".//p[contains(text(), '승진 기회')]/../div/ul").find_elements(By.XPATH, '//*[@id="ReviewCardSide"]/div[2]/ul/li[5]/div/ul/li[4]/span'))
promotion_opp = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '승진 기회')]/../div//span[contains(@style, 'width: 100%')]"
))
welfare_salary = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '복지/급여')]/../div//span[contains(@style, 'width: 100%')]"
))
work_life_balance = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '워라밸')]/../div//span[contains(@style, 'width: 100%')]"
))
culture = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '사내문화')]/../div//span[contains(@style, 'width: 100%')]"
))
management = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '경영진')]/../div//span[contains(@style, 'width: 100%')]"
))
print(promotion_opp, welfare_salary, work_life_balance, culture, management)
other_score = [promotion_opp, welfare_salary, work_life_balance, culture, management]

5 5 5 5 4


In [6]:
def extract_score_from_review(review):
    
    total_score_text = review.find_element(By.CSS_SELECTOR, r"#ReviewCardSide > div.mb-\[32px\] > span").get_attribute("innerHTML").strip()
    total_score = float(total_score_text)
    score_list = review.find_element(By.CSS_SELECTOR, "#ReviewCardSide > div:nth-child(2) > ul")

    def get_score(label):
        try:
            # li 중에서 label 텍스트가 포함된 p가 있는 li를 찾음
            li_tag = score_list.find_element(By.XPATH, f".//li[p[contains(text(), '{label}')]]")
            # 해당 li 안에서 div > ul로 이동
            ul = li_tag.find_element(By.XPATH, ".//div/ul")
            filled = ul.find_elements(By.XPATH, ".//li/span[contains(@style, 'width: 100%')]")
            return len(filled)
        except:
            return 0

    promotion_opp = get_score('승진 기회')
    welfare_salary = get_score('복지/급여')
    work_life_balance = get_score('워라밸')
    culture = get_score('사내문화')
    management = get_score('경영진')

    other_score = [promotion_opp, welfare_salary, work_life_balance, culture, management]
    return [total_score] + other_score

score_result = extract_score_from_review(review)
print(score_result)

[5.0, 5, 5, 5, 5, 4]


In [7]:
# 전체 페이지 수는 알 수 없음. 
# driver와 현재 url이 주어졌을 때 해당 url에 접속할 뿐인 get_url 만들기.
import time
def get_url(driver, url):
    while True:
        try:
            driver.get(url)
            driver.implicitly_wait(10) # 페이지가 모두 로딩될 때까지 최대 10초 기다리기.
            break
        except:
            driver.close()
            driver = webdriver.Chrome(service=service)
            time.sleep(60)

In [10]:
import pandas as pd
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "data", "종목별_잡플래닛_CHO.xlsx")
input_data = pd.read_excel(file_path)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

data = []

# 페이지 로딩 재시도 함수 만들기
def get_url(driver, url):
    while True:
        try: # 이하 코드를 시도해보세요.
            driver.get(url) # URL 이동
            driver.implicitly_wait(10) # 최대 10초 대가(로딩 기다림)
            break # get_url 성공시 break을 통해 while 빠져나감.
        except: # try에서 에러 발생시 이하 코드를 실행하세요.
            print("get_url 에러 발생, 재시도 중...")
            time.sleep(5)  # 단순히 쉬었다가 다시 driver.get(url)

for company, url in input_data[['종목명', 'URL']].values:
    print(f"===== {company} 크롤링 시작 =====")
    page = 1
    while True:
        current_url = f"{url}?page={page}"
        get_url(driver, current_url)
        print(f"현재 {company} - {page}페이지 크롤링 중...")
    
    # 팝업창 나타나면 닫기
        try:
            driver.find_element(By.CSS_SELECTOR, "#i8zsl").click() 
        except:
            pass

    # 현재 페이지에서 데이터 가져오기
    # 리뷰 리스트 가져오기
        review_list = driver.find_elements(By.CSS_SELECTOR, "#viewReviewsList > section")

        if not review_list:
            print(f"{company} - page {page}에서 리뷰 없음, 종료")
            break


        for review in review_list:
            try:
                date_container = review.find_element(By.CSS_SELECTOR, "div.relative.flex.items-center.justify-between")
                date_spans = date_container.find_elements(By.CSS_SELECTOR, "div.flex.items-center span.text-body2")
                date = date_spans[-1].get_attribute("innerHTML").replace("작성", "").strip()

                score = extract_score_from_review(review)
                record = [company, date] + score  # ✅ score로 바꿔야 맞음!
                data.append(record)
            except Exception as e:
                print(f"리뷰 파싱 오류: {e}")
                continue
            
        print(f"{company} - {page}페이지 크롤링 완료")
        page += 1 # 다음 페이지로 이동

df = pd.DataFrame(data, columns = ["기업", "날짜", "전체평점", "승진기회", "복지/급여", "워라밸", "사내문화", "경영진"])
display(df.head())

===== KB자산운용 크롤링 시작 =====
현재 KB자산운용 - 1페이지 크롤링 중...
KB자산운용 - 1페이지 크롤링 완료
현재 KB자산운용 - 2페이지 크롤링 중...
KB자산운용 - 2페이지 크롤링 완료
현재 KB자산운용 - 3페이지 크롤링 중...
KB자산운용 - 3페이지 크롤링 완료
현재 KB자산운용 - 4페이지 크롤링 중...
KB자산운용 - 4페이지 크롤링 완료
현재 KB자산운용 - 5페이지 크롤링 중...
KB자산운용 - 5페이지 크롤링 완료
현재 KB자산운용 - 6페이지 크롤링 중...
KB자산운용 - 6페이지 크롤링 완료
현재 KB자산운용 - 7페이지 크롤링 중...
KB자산운용 - 7페이지 크롤링 완료
현재 KB자산운용 - 8페이지 크롤링 중...
KB자산운용 - 8페이지 크롤링 완료
현재 KB자산운용 - 9페이지 크롤링 중...
KB자산운용 - 9페이지 크롤링 완료
현재 KB자산운용 - 10페이지 크롤링 중...
KB자산운용 - 10페이지 크롤링 완료
현재 KB자산운용 - 11페이지 크롤링 중...
KB자산운용 - page 11에서 리뷰 없음, 종료
===== 임팩트스퀘어 크롤링 시작 =====
현재 임팩트스퀘어 - 1페이지 크롤링 중...
임팩트스퀘어 - 1페이지 크롤링 완료
현재 임팩트스퀘어 - 2페이지 크롤링 중...
임팩트스퀘어 - 2페이지 크롤링 완료
현재 임팩트스퀘어 - 3페이지 크롤링 중...
임팩트스퀘어 - page 3에서 리뷰 없음, 종료


,기업,날짜,전체평점,승진기회,복지/급여,워라밸,사내문화,경영진
0,KB자산운용,2025. 03,2.0,2,2,3,2,1
1,KB자산운용,2025. 02,3.0,4,3,4,4,2
2,KB자산운용,2024. 11,3.0,4,2,2,4,3
3,KB자산운용,2024. 11,3.0,3,3,3,3,3
4,KB자산운용,2024. 08,1.0,1,1,1,1,1


In [11]:
print(df.dtypes)

기업        object
날짜        object
전체평점     float64
승진기회       int64
복지/급여      int64
워라밸        int64
사내문화       int64
경영진        int64
dtype: object


In [14]:
# 데이터 정제
stat = df.groupby(["기업", "날짜"], as_index = False)[["전체평점", "승진기회", "복지/급여", "워라밸", "사내문화", "경영진"]].mean()
stat.sort_values(by = ["기업", "날짜"], inplace = True)
display(stat.head(10))
display(stat)

,기업,날짜,전체평점,승진기회,복지/급여,워라밸,사내문화,경영진
0,KB자산운용,2016. 03,3.0,4.0,2.0,3.0,2.0,2.0
1,KB자산운용,2016. 05,3.5,3.5,3.5,3.5,3.5,3.5
2,KB자산운용,2016. 08,4.0,4.0,3.0,4.0,3.0,3.0
3,KB자산운용,2018. 05,4.0,3.0,3.0,3.0,3.0,3.0
4,KB자산운용,2019. 02,4.0,4.0,4.0,5.0,4.0,4.0
5,KB자산운용,2019. 03,3.0,2.0,3.0,3.0,3.0,3.0
6,KB자산운용,2019. 10,4.0,3.0,5.0,4.0,4.0,3.0
7,KB자산운용,2020. 05,4.0,4.0,4.0,5.0,4.0,4.0
8,KB자산운용,2020. 06,4.0,2.5,3.0,4.5,4.0,2.5
9,KB자산운용,2020. 12,4.0,3.0,3.0,4.0,4.0,3.0


,기업,날짜,전체평점,승진기회,복지/급여,워라밸,사내문화,경영진
0,KB자산운용,2016. 03,3.000000,4.000000,2.000000,3.000000,2.000000,2.000000
1,KB자산운용,2016. 05,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
2,KB자산운용,2016. 08,4.000000,4.000000,3.000000,4.000000,3.000000,3.000000
3,KB자산운용,2018. 05,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000
4,KB자산운용,2019. 02,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000
5,KB자산운용,2019. 03,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000
6,KB자산운용,2019. 10,4.000000,3.000000,5.000000,4.000000,4.000000,3.000000
7,KB자산운용,2020. 05,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000
8,KB자산운용,2020. 06,4.000000,2.500000,3.000000,4.500000,4.000000,2.500000
9,KB자산운용,2020. 12,4.000000,3.000000,3.000000,4.000000,4.000000,3.000000


In [17]:
stat.to_csv(os.path.join(base_dir, "기업월별평점.csv"), index=False, encoding = "euc-kr")